# Importance of the hospitality industry in Barcelona and Madrid - Notebook

## 1. Importing sources

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import requests
import urllib.request
import random
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

In [9]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         

typing-extensions-3. | 8 KB      | ##################################### | 100% 
libdeflate-1.7       | 67 KB     | ##################################### | 100% 
boto3-1.17.24        | 70 KB     | ##################################### | 100% 
markupsafe-1.1.1     | 27 KB     | ##################################### | 100% 
gst-plugins-base-1.1 | 2.5 MB    | ##################################### | 100% 
krb5-1.17.2          | 1.4 MB    | ##################################### | 100% 
testpath-0.4.4       | 85 KB     | ##################################### | 100% 
backports.functools_ | 8 KB      | ##################################### | 100% 
cloudpickle-1.6.0    | 22 KB     | ##################################### | 100% 
snappy-1.1.8         | 32 KB     | ##################################### | 100% 
numpy-1.20.1         | 5.8 MB    | ##################################### | 100% 
cytoolz-0.11.0       | 403 KB    | ##################################### | 100% 
attrs-20.3.0         | 41 KB

sleef-3.5.1          | 1.5 MB    | ##################################### | 100% 
mpmath-1.2.1         | 437 KB    | ##################################### | 100% 
notebook-6.2.0       | 6.2 MB    | ##################################### | 100% 
libwebp-base-1.2.0   | 808 KB    | ##################################### | 100% 
qt-5.12.9            | 99.5 MB   | ##################################### | 100% 
blas-1.0             | 1 KB      | ##################################### | 100% 
mpfr-4.0.2           | 648 KB    | ##################################### | 100% 
requests-oauthlib-1. | 21 KB     | ##################################### | 100% 
_openmp_mutex-4.5    | 5 KB      | ##################################### | 100% 
libiconv-1.16        | 1.4 MB    | ##################################### | 100% 
pcre-8.44            | 261 KB    | ##################################### | 100% 
lxml-4.6.2           | 1.5 MB    | ##################################### | 100% 
ibm-wsrt-py37main-ma | 2 KB 

mysql-libs-8.0.23    | 1.8 MB    | ##################################### | 100% 
_pytorch_select-0.2  | 2 KB      | ##################################### | 100% 
libxgboost-1.3.3     | 3.3 MB    | ##################################### | 100% 
botocore-1.20.24     | 4.5 MB    | ##################################### | 100% 
_libgcc_mutex-0.1    | 3 KB      | ##################################### | 100% 
unixodbc-2.3.9       | 293 KB    | ##################################### | 100% 
bleach-3.3.0         | 111 KB    | ##################################### | 100% 
pexpect-4.8.0        | 47 KB     | ##################################### | 100% 
blosc-1.21.0         | 841 KB    | ##################################### | 100% 
libpng-1.6.37        | 306 KB    | ##################################### | 100% 
prometheus_client-0. | 44 KB     | ##################################### | 100% 
pyshp-2.1.0          | 31 KB     | ##################################### | 100% 
tabulate-0.8.9       | 26 KB

done


In [10]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1902
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1902
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    -

## 2. Analysing Barcelona

### 2.1. Download and scrape the list of districts

In [2]:
url = "https://es.wikipedia.org/wiki/Distritos_de_Barcelona"
page = urllib.request.urlopen(url)

In [3]:
soup = BeautifulSoup(page, "lxml")

In [4]:
all_tables = soup.find_all("table")

In [5]:
right_table = soup.find_all('table', class_='wikitable')
right_table[0]

<table border="1" class="wikitable">
<tbody><tr bgcolor="silver">
<th width="23"><center><b>Nº</b>
</center></th>
<th width="115"><center><b> Distrito </b></center>
</th>
<th><center><b>Imagen</b></center>
</th>
<th><center><b>Superficie km²<sup class="reference separada" id="cite_ref-1"><a href="#cite_note-1"><span class="corchete-llamada">[</span>1<span class="corchete-llamada">]</span></a></sup>​</b></center>
</th>
<th><center><b>Población (2019)<sup class="reference separada" id="cite_ref-2"><a href="#cite_note-2"><span class="corchete-llamada">[</span>2<span class="corchete-llamada">]</span></a></sup>​</b></center>
</th>
<th><center><b> Densidad hab/km²</b></center>
</th>
<th><center><b> Barrios (nº)</b></center>
</th>
<th><center><b> Regidor </b></center>
</th></tr>
<tr>
<td align="center">1
</td>
<td align="left"><center> <b><a href="/wiki/Distrito_de_Ciudad_Vieja_(Barcelona)" title="Distrito de Ciudad Vieja (Barcelona)">Ciutat Vella</a></b>
</center></td>
<td align="center"><a 

In [6]:
column_names = ['Number','District','Location','Size (km2)','Population','Density','Neighborhoods','Council']
bcn = pd.DataFrame(columns =column_names)

In [7]:
for tr_cell in right_table[0].find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==8:
        bcn.loc[len(bcn)] = row_data

In [8]:
bcn

,Number,District,Location,Size (km2),Population,Density,Neighborhoods,Council
0,1,Ciutat Vella,,"4,11",103 429,"25 159,09","El Raval (1), Barrio Gótico (2), La Barcelonet...",Jordi Rabassa (Barcelona en Comú)
1,2,Eixample,,"7,46",265 910,"35 625,67","El Fort Pienc (5), Sagrada Familia (6), Dreta ...",Jordi Martí (Barcelona en Comú)
2,3,Sants-Montjuïc,,"22,68",184 091,"8118,32","Poble Sec (11), La Marina del Prat Vermell (12...",Marc Serra (Barcelona en Comú)
3,4,Les Corts,,"6,02",81 974,"13 607,11","Les Corts (19), La Maternitat i Sant Ramon (2...",Xavier Marcé (PSC)
4,5,Sarrià-Sant Gervasi,,"19,91",149 260,"7497,11","Vallvidrera, el Tibidabo i les Planes (22), Sa...",Albert Batlle (PSC)
5,6,Gràcia,,"4,19",121 798,"29 082,62","Vallcarca i els Penitents (28), El Coll (29), ...",Eloi Badia (Barcelona en Comú)
6,7,Horta - Guinardó,,"11,96",171 495,"14 342,64","Baix Guinardó (33), Can Baró (34), El Guinardó...",Rosa Alarcón (PSC)
7,8,Nou Barris,,"8,05",170 669,"21 198,48","Vilapicina i La Torre Llobeta (44), Porta (45)...",Marga Marí-Klose (PSC)
8,9,Sant Andreu,,"6,59",149 821,"22 724,25","La Trinitat Vella (57), Baró de Viver (58), El...",Lucía Martín (Barcelona en Comú)
9,10,Sant Martí,,"10,39",238 315,"22 943,58","El Camp de l'Arpa del Clot (64), El Clot (65),...",David Escudé (PSC)


### 2.2. Data cleaning

In [11]:
bcn_districts = bcn.drop(['Number','Location','Density','Council'], axis=1)

In [12]:
bcn_districts.District = bcn_districts.District.replace({"Gràcia": "Gracia Barcelona"})

In [13]:
bcn_districts

,District,Size (km2),Population,Neighborhoods
0,Ciutat Vella,"4,11",103 429,"El Raval (1), Barrio Gótico (2), La Barcelonet..."
1,Eixample,"7,46",265 910,"El Fort Pienc (5), Sagrada Familia (6), Dreta ..."
2,Sants-Montjuïc,"22,68",184 091,"Poble Sec (11), La Marina del Prat Vermell (12..."
3,Les Corts,"6,02",81 974,"Les Corts (19), La Maternitat i Sant Ramon (2..."
4,Sarrià-Sant Gervasi,"19,91",149 260,"Vallvidrera, el Tibidabo i les Planes (22), Sa..."
5,Gracia Barcelona,"4,19",121 798,"Vallcarca i els Penitents (28), El Coll (29), ..."
6,Horta - Guinardó,"11,96",171 495,"Baix Guinardó (33), Can Baró (34), El Guinardó..."
7,Nou Barris,"8,05",170 669,"Vilapicina i La Torre Llobeta (44), Porta (45)..."
8,Sant Andreu,"6,59",149 821,"La Trinitat Vella (57), Baró de Viver (58), El..."
9,Sant Martí,"10,39",238 315,"El Camp de l'Arpa del Clot (64), El Clot (65),..."


In [15]:
bcn_districts['Latitude'] = bcn_districts['District'].apply(lambda x: geolocator.geocode(x).latitude)
bcn_districts['Longitude'] = bcn_districts['District'].apply(lambda x: geolocator.geocode(x).longitude)

In [16]:
bcn_districts

,District,Size (km2),Population,Neighborhoods,Latitude,Longitude
0,Ciutat Vella,"4,11",103 429,"El Raval (1), Barrio Gótico (2), La Barcelonet...",41.374985,2.173277
1,Eixample,"7,46",265 910,"El Fort Pienc (5), Sagrada Familia (6), Dreta ...",41.393689,2.163655
2,Sants-Montjuïc,"22,68",184 091,"Poble Sec (11), La Marina del Prat Vermell (12...",41.364762,2.154233
3,Les Corts,"6,02",81 974,"Les Corts (19), La Maternitat i Sant Ramon (2...",41.385244,2.132863
4,Sarrià-Sant Gervasi,"19,91",149 260,"Vallvidrera, el Tibidabo i les Planes (22), Sa...",41.413039,2.107620
5,Gracia Barcelona,"4,19",121 798,"Vallcarca i els Penitents (28), El Coll (29), ...",41.410171,2.155136
6,Horta - Guinardó,"11,96",171 495,"Baix Guinardó (33), Can Baró (34), El Guinardó...",41.428556,2.143617
7,Nou Barris,"8,05",170 669,"Vilapicina i La Torre Llobeta (44), Porta (45)...",41.445815,2.179801
8,Sant Andreu,"6,59",149 821,"La Trinitat Vella (57), Baró de Viver (58), El...",41.437439,2.196859
9,Sant Martí,"10,39",238 315,"El Camp de l'Arpa del Clot (64), El Clot (65),...",41.406782,2.203655


In [17]:
bcn_districts.shape

(10, 6)

### 2.3. Mapping districts

In [18]:
barcelona = 'Barcelona, Barcelona'

geolocator = Nominatim(user_agent="Micar_21")
location = geolocator.geocode(barcelona)
latitude = location.latitude
longitude = location.longitude

print('The geographycal coordinate of Barcelona are {},{}.'.format(latitude,longitude))

The geographycal coordinate of Barcelona are 41.3828939,2.1774322.


In [19]:
map_bcn = folium.Map(location=[latitude, longitude], zoom_start = 10)

for lat, lng, district, neighborhood in zip(bcn_districts['Latitude'], bcn_districts['Longitude'], bcn_districts['District'], bcn_districts['Neighborhoods']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_bcn)

map_bcn

### 2.4. Define Foursquare credentials and version

In [20]:
CLIENT_ID = 'TILJIOO41TAHQHNOEJCU3BQJ5D4PHGUEXCPT0LKQNRJWSPYO' # your Foursquare ID
CLIENT_SECRET = 'P4QZGN4PGDZPXOYD55R2NBWVENSPLM3K4JB4YNZXZLC0K4GQ' # your Foursquare Secret
ACCESS_TOKEN = 'EVSQ1WHTTHP1S5G2UZNCCR0OPFR2V5IQIHLTJNYBGEHB1W0J' # your FourSquare Access Token
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TILJIOO41TAHQHNOEJCU3BQJ5D4PHGUEXCPT0LKQNRJWSPYO
CLIENT_SECRET:P4QZGN4PGDZPXOYD55R2NBWVENSPLM3K4JB4YNZXZLC0K4GQ


### 2.5. Exploring venues in a district 

In [21]:
#Explore the first district - get the name

bcn_districts.loc[0, 'District']

'Ciutat Vella'

In [22]:
district_latitude = bcn_districts.loc[0, 'Latitude']
district_longitude = bcn_districts.loc[0, 'Longitude']
district_name = bcn_districts.loc[0, 'District']

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, district_latitude, 
                                                               district_longitude))


Latitude and longitude values of Ciutat Vella are 41.3749846, 2.17327724224704.


In [23]:
LIMIT = 100
radius = 5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius{}&limit{}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION,district_latitude, district_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TILJIOO41TAHQHNOEJCU3BQJ5D4PHGUEXCPT0LKQNRJWSPYO&client_secret=P4QZGN4PGDZPXOYD55R2NBWVENSPLM3K4JB4YNZXZLC0K4GQ&v=20180604&ll=41.3749846,2.17327724224704&radius5000&limit100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6049455c31e8773ab7103ba5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 809,
  'headerLocation': 'El Raval',
  'headerFullLocation': 'El Raval, Barcelona',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 153,
  'suggestedBounds': {'ne': {'lat': 41.37959095768437,
    'lng': 2.178192212980128},
   'sw': {'lat': 41.369967470031725, 'lng': 2.167219880046316}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '58c2c0a49465dd5494bb118f',
       'name': 'macera',
       'location': {'address': 'Carrer Nou de la Rambla 102',
        'lat': 41.375589194805286,
        'lng': 2.17049320007949,
        'labeledLatLngs': [{'label': 'disp

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


AttributeError: 'Series' object has no attribute '_mgr'

In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


### 2.6. Exploring venues in each district

In [30]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius{}&limit{}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius,
        LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
bcn_venues = getNearbyVenues(names=bcn_districts['District'], latitudes=bcn_districts['Latitude'], longitudes=bcn_districts['Longitude'])

Ciutat Vella
Eixample
Sants-Montjuïc
Les Corts
Sarrià-Sant Gervasi
Gracia Barcelona
Horta - Guinardó
Nou Barris
Sant Andreu
Sant Martí


In [32]:
print(bcn_venues.shape)
bcn_venues.head()

(300, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ciutat Vella,41.374985,2.173277,macera,41.375589,2.170493,Cocktail Bar
1,Ciutat Vella,41.374985,2.173277,The Fish & Chips Shop,41.375965,2.174152,Bistro
2,Ciutat Vella,41.374985,2.173277,Cassette Bar,41.377324,2.173629,Bar
3,Ciutat Vella,41.374985,2.173277,Marea Alta,41.376484,2.175106,Seafood Restaurant
4,Ciutat Vella,41.374985,2.173277,Pizza Circus,41.377905,2.172911,Pizza Place


In [39]:
bcn_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Ciutat Vella,30,30,30,30,30,30
Eixample,30,30,30,30,30,30
Gracia Barcelona,30,30,30,30,30,30
Horta - Guinardó,30,30,30,30,30,30
Les Corts,30,30,30,30,30,30
Nou Barris,30,30,30,30,30,30
Sant Andreu,30,30,30,30,30,30
Sant Martí,30,30,30,30,30,30
Sants-Montjuïc,30,30,30,30,30,30


In [62]:
bcn_venues

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ciutat Vella,41.374985,2.173277,macera,41.375589,2.170493,Cocktail Bar
1,Ciutat Vella,41.374985,2.173277,The Fish & Chips Shop,41.375965,2.174152,Bistro
2,Ciutat Vella,41.374985,2.173277,Cassette Bar,41.377324,2.173629,Bar
3,Ciutat Vella,41.374985,2.173277,Marea Alta,41.376484,2.175106,Seafood Restaurant
4,Ciutat Vella,41.374985,2.173277,Pizza Circus,41.377905,2.172911,Pizza Place
5,Ciutat Vella,41.374985,2.173277,Sala Apolo,41.374355,2.169668,Concert Hall
6,Ciutat Vella,41.374985,2.173277,Miscelanea Gallery-Shop-Café,41.377702,2.173564,Art Gallery
7,Ciutat Vella,41.374985,2.173277,El Pachuco,41.376369,2.169148,Mexican Restaurant
8,Ciutat Vella,41.374985,2.173277,Museu Marítim de Barcelona,41.375789,2.175841,Museum
9,Ciutat Vella,41.374985,2.173277,My Fu*king Restaurant,41.377767,2.173236,Spanish Restaurant


### 2.7. Grouping venues by category

In [68]:
venue_category_bcn = bcn_venues.groupby('Venue Category').count() 
venue_category_bcn

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Argentinian Restaurant,3,3,3,3,3,3
Art Gallery,2,2,2,2,2,2
Art Museum,3,3,3,3,3,3
Asian Restaurant,1,1,1,1,1,1
Bakery,7,7,7,7,7,7
Bar,6,6,6,6,6,6
Basketball Stadium,1,1,1,1,1,1
Bed & Breakfast,1,1,1,1,1,1
Beer Bar,3,3,3,3,3,3


### 2.8. Exploring hospitality venues

In [110]:
hospitality_bcn = bcn_venues[bcn_venues['Venue Category'].isin(['Argentinian Restaurant','Asian Restaurant','Bakery','Bar','Bed & Breakfast','Beer Bar','Bistro','Breakfast Spot','Burger Joint','Café','Chinese Restaurant','Cocktail Bar','Coffee Shop','Cupcake Shop','Deli/Bodega','Dessert Shop','Diner','Donut Shop','Empanada Restaurant','Food','Food & Drink Shop','Gastropub','German Restaurant','Hotel','Ice Cream Shop','Italian Restaurant','Japanese Restaurant','Mediterranean Restaurant','Mexican Restaurant','Pizza Place','Polish Restaurant','Restaurant','Sandwich Place','Seafood Restaurant','Snack Placce','Spanish Restaurant','Sushi Restaurant','Taco Place','Tapas Restaurant','Thai Restaurant','Vegetarian/Vegan Restaurant','Wine Bar'])]
hospitality_bcn

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ciutat Vella,41.374985,2.173277,macera,41.375589,2.170493,Cocktail Bar
1,Ciutat Vella,41.374985,2.173277,The Fish & Chips Shop,41.375965,2.174152,Bistro
2,Ciutat Vella,41.374985,2.173277,Cassette Bar,41.377324,2.173629,Bar
3,Ciutat Vella,41.374985,2.173277,Marea Alta,41.376484,2.175106,Seafood Restaurant
4,Ciutat Vella,41.374985,2.173277,Pizza Circus,41.377905,2.172911,Pizza Place
7,Ciutat Vella,41.374985,2.173277,El Pachuco,41.376369,2.169148,Mexican Restaurant
9,Ciutat Vella,41.374985,2.173277,My Fu*king Restaurant,41.377767,2.173236,Spanish Restaurant
10,Ciutat Vella,41.374985,2.173277,Frankie Gallo Cha Cha Cha,41.378450,2.172683,Pizza Place
16,Ciutat Vella,41.374985,2.173277,Cañete,41.379154,2.173092,Tapas Restaurant
17,Ciutat Vella,41.374985,2.173277,Trópico,41.377817,2.171247,Restaurant


In [113]:
hospitality_bcn.shape

(145, 7)

In [150]:
hospitality_districts_bcn = hospitality_bcn.groupby('District').count()
hospitality_districts_bcn = hospitality_districts_bcn.drop(['District Latitude','District Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
hospitality_districts_bcn = hospitality_districts_bcn.sort_values(by=['Venue'], ascending=[False])
hospitality_districts_bcn

,Venue
District,
Sant Martí,24
Les Corts,20
Ciutat Vella,18
Eixample,17
Gracia Barcelona,17
Horta - Guinardó,14
Nou Barris,13
Sant Andreu,13
Sarrià-Sant Gervasi,7


In [149]:
hospitality_categories_bcn = hospitality_bcn.groupby('Venue Category').count()
hospitality_categories_bcn = hospitality_categories_bcn.drop(['District','District Latitude','District Longitude','Venue Latitude','Venue Longitude'], axis=1)
hospitality_categories_bcn = hospitality_categories_bcn.sort_values(by=['Venue'], ascending=[False])
hospitality_categories_bcn

,Venue
Venue Category,
Restaurant,16
Tapas Restaurant,15
Spanish Restaurant,14
Mediterranean Restaurant,13
Hotel,9
Bakery,7
Bar,6
Pizza Place,6
Italian Restaurant,5


## 3.  Analysing Madrid

### 3.1. Download and scrape the list of districts

In [81]:
url_2 = "https://en.wikipedia.org/wiki/Districts_of_Madrid"
page_2 = urllib.request.urlopen(url_2)

In [82]:
soup_2 = BeautifulSoup(page_2, "lxml")

In [83]:
all_tables_2 = soup_2.find_all("table")

In [84]:
right_table_2 = soup_2.find_all('table', class_='wikitable sortable')
right_table_2[0]

<table class="wikitable sortable">
<tbody><tr>
<th>District Number
</th>
<th>Name
</th>
<th>District area<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[n 1]</a></sup><br/> (Ha.)
</th>
<th>Population
</th>
<th>Population density<br/>(Hab./Ha.)
</th>
<th>Location
</th>
<th>Administrative wards
</th></tr>
<tr>
<td align="center">1
</td>
<td align="center"><a href="/wiki/Centro_(Madrid)" title="Centro (Madrid)">Centro</a>
</td>
<td align="right"><span data-sort-value="7002522820000000000♠">522.82</span>
</td>
<td align="right"><span data-sort-value="7005131928000000000♠">131,928</span>
</td>
<td align="right"><span data-sort-value="7002252340000000000♠">252.34</span>
</td>
<td align="center"><a class="image" href="/wiki/File:Centro_District_loc-map.svg"><img alt="Centro District loc-map.svg" data-file-height="527" data-file-width="527" decoding="async" height="200" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/09/Centro_District_loc-map.svg/200px-Centro_District_loc

In [85]:
column_names_2 = ['Number','District','Size (ha)','Population','Density','Location','Neighborhoods']
mad = pd.DataFrame(columns =column_names_2)

In [86]:
for tr_cell in right_table_2[0].find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==7:
        mad.loc[len(mad)] = row_data

In [87]:
mad

,Number,District,Size (ha),Population,Density,Location,Neighborhoods
0,1,Centro,522.82,"131,928",252.34,,Palacio (11)Embajadores (12)Cortes (13)Justici...
1,2,Arganzuela,646.22,"151,965",235.16,,Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,3,Retiro,546.62,"118,516",216.82,,Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...
3,4,Salamanca,539.24,"143,800",266.67,,Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,5,Chamartín,917.55,"143,424",156.31,,El Viso (51)Prosperidad (52)Ciudad Jardín (53)...
5,6,Tetuán,537.47,"153,789",286.13,,Bellas Vistas (61)Cuatro Caminos (62)Castillej...
6,7,Chamberí,467.92,"137,401",293.64,,Gaztambide (71)Arapiles (72)Trafalgar (73)Alma...
7,8,Fuencarral-El Pardo,"23,783.84","238,756",10.04,,El Pardo (81)Fuentelarreina (82)Peñagrande (83...
8,9,Moncloa-Aravaca,"4,653.11","116,903",25.12,,Casa de Campo (91)Argüelles (92)Ciudad Univers...
9,10,Latina,"2,542.72","233,808",91.95,,Los Cármenes (101)Puerta del Ángel (102)Lucero...


### 3.2. Data cleaning

In [88]:
mad_districts = mad.drop(['Number','Location','Density'], axis=1)

In [89]:
mad_districts.District = mad_districts.District.replace({"Retiro": "Retiro Madrid"})
mad_districts.District = mad_districts.District.replace({"Tetuán": "Tetuán Madrid"})

In [90]:
mad_districts

,District,Size (ha),Population,Neighborhoods
0,Centro,522.82,"131,928",Palacio (11)Embajadores (12)Cortes (13)Justici...
1,Arganzuela,646.22,"151,965",Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,Retiro Madrid,546.62,"118,516",Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...
3,Salamanca,539.24,"143,800",Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,Chamartín,917.55,"143,424",El Viso (51)Prosperidad (52)Ciudad Jardín (53)...
5,Tetuán Madrid,537.47,"153,789",Bellas Vistas (61)Cuatro Caminos (62)Castillej...
6,Chamberí,467.92,"137,401",Gaztambide (71)Arapiles (72)Trafalgar (73)Alma...
7,Fuencarral-El Pardo,"23,783.84","238,756",El Pardo (81)Fuentelarreina (82)Peñagrande (83...
8,Moncloa-Aravaca,"4,653.11","116,903",Casa de Campo (91)Argüelles (92)Ciudad Univers...
9,Latina,"2,542.72","233,808",Los Cármenes (101)Puerta del Ángel (102)Lucero...


In [91]:
mad_districts['Latitude'] = mad_districts['District'].apply(lambda x: geolocator.geocode(x).latitude)
mad_districts['Longitude'] = mad_districts['District'].apply(lambda x: geolocator.geocode(x).longitude)

In [92]:
mad_districts

,District,Size (ha),Population,Neighborhoods,Latitude,Longitude
0,Centro,522.82,"131,928",Palacio (11)Embajadores (12)Cortes (13)Justici...,47.549025,1.732406
1,Arganzuela,646.22,"151,965",Imperial (21)Acacias (22)Chopera (23)Legazpi (...,40.398068,-3.693734
2,Retiro Madrid,546.62,"118,516",Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...,40.411150,-3.676057
3,Salamanca,539.24,"143,800",Recoletos (41)Goya (42)Fuente del Berro (43)Gu...,40.965157,-5.664018
4,Chamartín,917.55,"143,424",El Viso (51)Prosperidad (52)Ciudad Jardín (53)...,40.701869,-4.957008
5,Tetuán Madrid,537.47,"153,789",Bellas Vistas (61)Cuatro Caminos (62)Castillej...,40.460578,-3.698281
6,Chamberí,467.92,"137,401",Gaztambide (71)Arapiles (72)Trafalgar (73)Alma...,45.566267,5.920364
7,Fuencarral-El Pardo,"23,783.84","238,756",El Pardo (81)Fuentelarreina (82)Peñagrande (83...,40.556346,-3.778591
8,Moncloa-Aravaca,"4,653.11","116,903",Casa de Campo (91)Argüelles (92)Ciudad Univers...,40.439495,-3.744204
9,Latina,"2,542.72","233,808",Los Cármenes (101)Puerta del Ángel (102)Lucero...,41.459526,13.012591


In [93]:
mad_districts.shape

(21, 6)

### 3.3. Mapping districts

In [94]:
madrid = 'Madrid, Madrid'

geolocator_2 = Nominatim(user_agent="Micar_21")
location_2 = geolocator_2.geocode(madrid)
latitude_2 = location_2.latitude
longitude_2 = location_2.longitude

print('The geographycal coordinate of Madrid are {},{}.'.format(latitude_2,longitude_2))

The geographycal coordinate of Madrid are 40.4167047,-3.7035825.


In [95]:
map_mad = folium.Map(location=[latitude_2, longitude_2], zoom_start = 10)

for lat, lng, district, neighborhood in zip(mad_districts['Latitude'], mad_districts['Longitude'], mad_districts['District'], mad_districts['Neighborhoods']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_mad)

map_mad

### 3.4. Exploring venues in a district 

In [96]:
#Explore the first district - get the name

mad_districts.loc[0, 'District']

'Centro'

In [97]:
district_latitude_2 = mad_districts.loc[0, 'Latitude']
district_longitude_2 = mad_districts.loc[0, 'Longitude']
district_name_2 = mad_districts.loc[0, 'District']

print('Latitude and longitude values of {} are {}, {}.'.format(district_name_2, district_latitude_2, 
                                                               district_longitude_2))


Latitude and longitude values of Centro are 47.5490251, 1.7324062.


In [98]:
LIMIT =100
radius = 5000
url_2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius{}&limit{}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION,district_latitude_2, district_longitude_2, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TILJIOO41TAHQHNOEJCU3BQJ5D4PHGUEXCPT0LKQNRJWSPYO&client_secret=P4QZGN4PGDZPXOYD55R2NBWVENSPLM3K4JB4YNZXZLC0K4GQ&v=20180604&ll=41.3749846,2.17327724224704&radius5000&limit100'

In [99]:
results_2 = requests.get(url_2).json()
results_2

{'meta': {'code': 200, 'requestId': '60495a8a222e3f1aecf967c1'},
 'response': {'suggestedRadius': 10000,
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 47.61630554199219,
    'lng': 1.8611417027702557},
   'sw': {'lat': 47.53214721679687, 'lng': 1.5960114900441842}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52713fa2498e754b8f085c5a',
       'name': 'Zara',
       'location': {'address': 'Sahara Center',
        'lat': 47.6122534,
        'lng': 1.7306192,
        'labeledLatLngs': [{'label': 'display',
          'lat': 47.6122534,
          'lng': 1.7306192}],
        'distance': 7039,
        'cc': 'FR',
        'country': 'Franc

In [100]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [101]:
venues_2 = results_2['response']['groups'][0]['items']
    
nearby_venues_2 = json_normalize(venues_2) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_2 =nearby_venues_2.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_2['venue.categories'] = nearby_venues_2.apply(get_category_type, axis=1)

# clean columns
nearby_venues_2.columns = [col.split(".")[-1] for col in nearby_venues_2.columns]

nearby_venues_2.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


AttributeError: 'Series' object has no attribute '_mgr'

In [102]:
print('{} venues were returned by Foursquare.'.format(nearby_venues_2.shape[0]))

7 venues were returned by Foursquare.


### 3.5. Exploring venues in each district

In [103]:
def getNearbyVenues_2(names, latitudes, longitudes):
    
    venues_list_2=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url_2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius{}&limit{}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius,
        LIMIT)
            
        # make the GET request
        results_2 = requests.get(url_2).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list_2.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_2])

    nearby_venues_2 = pd.DataFrame([item for venue_list_2 in venues_list_2 for item in venue_list_2])
    nearby_venues_2.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_2)

In [104]:
mad_venues = getNearbyVenues_2(names=mad_districts['District'], latitudes=mad_districts['Latitude'], longitudes=mad_districts['Longitude'])

Centro
Arganzuela
Retiro Madrid
Salamanca
Chamartín
Tetuán Madrid
Chamberí
Fuencarral-El Pardo
Moncloa-Aravaca
Latina
Carabanchel
Usera
Puente de Vallecas
Moratalaz
Ciudad Lineal
Hortaleza
Villaverde
Villa de Vallecas
Vicálvaro
San Blas-Canillejas
Barajas


In [105]:
print(mad_venues.shape)
mad_venues.head()

(584, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centro,47.549025,1.732406,Zara,47.612253,1.730619,Clothing Store
1,Centro,47.549025,1.732406,Precy Anthony,47.537748,1.812981,Construction & Landscaping
2,Centro,47.549025,1.732406,Bath & Body Works,47.612253,1.730619,Cosmetics Shop
3,Centro,47.549025,1.732406,cafe friends,47.612480,1.730613,Café
4,Centro,47.549025,1.732406,Alimentation Générale,47.542476,1.848846,Supermarket


In [106]:
mad_venues.groupby('District').count()

,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Arganzuela,30,30,30,30,30,30
Barajas,30,30,30,30,30,30
Carabanchel,30,30,30,30,30,30
Centro,7,7,7,7,7,7
Chamartín,7,7,7,7,7,7
Chamberí,30,30,30,30,30,30
Ciudad Lineal,30,30,30,30,30,30
Fuencarral-El Pardo,30,30,30,30,30,30
Hortaleza,30,30,30,30,30,30


In [107]:
mad_venues

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centro,47.549025,1.732406,Zara,47.612253,1.730619,Clothing Store
1,Centro,47.549025,1.732406,Precy Anthony,47.537748,1.812981,Construction & Landscaping
2,Centro,47.549025,1.732406,Bath & Body Works,47.612253,1.730619,Cosmetics Shop
3,Centro,47.549025,1.732406,cafe friends,47.612480,1.730613,Café
4,Centro,47.549025,1.732406,Alimentation Générale,47.542476,1.848846,Supermarket
5,Centro,47.549025,1.732406,Le Beauharnais,47.542572,1.849090,Restaurant
6,Centro,47.549025,1.732406,Le Col Vert,47.535973,1.608063,Pub
7,Arganzuela,40.398068,-3.693734,Tres Cerditos,40.397316,-3.694184,Chinese Restaurant
8,Arganzuela,40.398068,-3.693734,Mercado de Motores,40.399149,-3.691978,Flea Market
9,Arganzuela,40.398068,-3.693734,Museo del Ferrocarril (Antigua Estación de Del...,40.399395,-3.692286,Museum


### 3.6. Grouping venus by category

In [108]:
venue_category_mad = mad_venues.groupby('Venue Category').count() 
venue_category_mad

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,1,1,1,1,1,1
Airport,1,1,1,1,1,1
Airport Lounge,1,1,1,1,1,1
Airport Service,1,1,1,1,1,1
Argentinian Restaurant,7,7,7,7,7,7
Art Gallery,3,3,3,3,3,3
Art Museum,4,4,4,4,4,4
Art Studio,1,1,1,1,1,1
Asian Restaurant,5,5,5,5,5,5


### 3.7. Exploring hospitality venues

In [109]:
hospitality_mad = mad_venues[mad_venues['Venue Category'].isin(['Argentinian Restaurant','Asian Restaurant','BBQ Joint','Bakery','Bar','Beer Bar','Beer Garden','Bistro','Brazilian Restaurant','Breakfast Spot','Burger Joint','Café','Chinese Restaurant','Cocktail Bar','Coffee Shop','Cuban Restaurant','Cupcake Shop','Deli/Bodega','Dessert Shop','Diner','Donut Shop','Dumpling Restaurant','Eastern European Restaurant','Fast Food Restaurant','Food Service','French Restaurant','Gastropub','Hotel','Hotel Bar','Ice Cream Shop','Indian Restaurant','Italian Restaurant','Japanese Restaurant','Juice Bar','Korean Restaurant','Latin American Restaurant','Mediterranean Restaurant','Mexican Restaurant','Middle Eastern Restaurant','Paella Restaurant','Pastry Shop','Persian Restaurant','Peruvian Restaurant','Pizza Place','Polish Restaurant','Restaurant','Sandwich Place','Seafood Restaurant','Snack Placce','Soup Place','Spanish Restaurant','Steakhouse','Sushi Restaurant','Tapas Restaurant','Thai Restaurant','Vegetarian/Vegan Restaurant','Venezuelan Restaurant','Wine Bar'])]
hospitality_mad

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,Centro,47.549025,1.732406,cafe friends,47.612480,1.730613,Café
5,Centro,47.549025,1.732406,Le Beauharnais,47.542572,1.849090,Restaurant
7,Arganzuela,40.398068,-3.693734,Tres Cerditos,40.397316,-3.694184,Chinese Restaurant
10,Arganzuela,40.398068,-3.693734,Magasand Deli,40.396811,-3.691293,Restaurant
11,Arganzuela,40.398068,-3.693734,PanArte,40.399279,-3.694182,Bakery
12,Arganzuela,40.398068,-3.693734,Las tinajas,40.396993,-3.697779,Tapas Restaurant
14,Arganzuela,40.398068,-3.693734,Salón de Té Al Yabal,40.399015,-3.700249,Cocktail Bar
15,Arganzuela,40.398068,-3.693734,Trattoria In Crescendo,40.394582,-3.698388,Italian Restaurant
16,Arganzuela,40.398068,-3.693734,Havana Blues,40.402050,-3.698488,Cuban Restaurant
17,Arganzuela,40.398068,-3.693734,La Pequeña Graná,40.399574,-3.698550,Tapas Restaurant


In [114]:
hospitality_mad.shape

(319, 7)

In [147]:
hospitality_districts_mad = hospitality_mad.groupby('District').count()
hospitality_districts_mad = hospitality_districts_mad.drop(['District Latitude','District Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis=1)
hospitality_districts_mad = hospitality_districts_mad.sort_values(by=['Venue'], ascending=[False])
hospitality_districts_mad

,Venue
District,
Tetuán Madrid,23
Hortaleza,23
Ciudad Lineal,22
Latina,21
San Blas-Canillejas,19
Barajas,19
Salamanca,18
Carabanchel,17
Fuencarral-El Pardo,17


In [148]:
hospitality_categories_mad = hospitality_mad.groupby('Venue Category').count()
hospitality_categories_mad = hospitality_categories_mad.drop(['District','District Latitude','District Longitude','Venue Latitude','Venue Longitude'], axis=1)
hospitality_categories_mad = hospitality_categories_mad.sort_values(by=['Venue'], ascending=[False])
hospitality_categories_mad

,Venue
Venue Category,
Spanish Restaurant,61
Restaurant,21
Bar,17
Tapas Restaurant,17
Hotel,17
Italian Restaurant,16
Café,13
Pizza Place,12
Ice Cream Shop,11
